In [ ]:
import numpy as np
import pandas as pd

# Our functions
import EvalFunctions
import RecAlgs

The parameters to select how to tun the algorithm

In [1]:
# General
AmountOfRecommendations = 0     # The amount of 'good' recommendation that will be returned
TimeCutOffDays = 3              # How old the articles can be that we would consider recommending (older than X days are not considered)

# Data selection                
TrainingDataStartDate = 0       # From what day we want to collect data
TrainingDataWindowSize = 2      # How many days of training data we want
TestDataWindowSize = 1          # How many days of test data we want

# Algorithm specifics 
TypeOfRecAlg = 0                # Which RecAlg we want to use 0-Pop, 1-Rand, 2-CBF, 3-CF, 4-Hybrid

# Popular Baseline
TimePenaltyPerHour = 0.1        # The percentage on penalty per hour the news gets
TimePenaltyStart = 24           # After howmany hours in the past the penalty starts

# Random Baseline
MinScore = 0                    
MaxScore = 1

# Content based filtering

# Collaborative filtering

# Hybrid
UsePopBaseline = False
UseRandBaseLine = False
UseCBF = True
UseCF = True
TakeMax = False
Weights = [0.2, 0.4, 0.4]


Data selection

In [ ]:
AllTrainingData = 